In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import d_utils as d

/users/jtmyllar/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_hip.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
DATASET_PATH = 'mvtec_data'
DATA_CATEGORIES = d.MVTEC_ITEMTYPES

In [7]:
# As a baseline, evaluate an STM with no discrepancy scaling.
# In the load_STM function, scaling is disabled by not entering a category.
# Calculating PRO is very slow, so it is disabled here.

aucs = []
img_aucs = []
pros = []

for category in d.MVTEC_ITEMTYPES:
  args = d.DummyArgs()
  args.category = category
  args.scaling = False
  args.pro = False
  args.data_dir = DATASET_PATH
  args.batch_size = 32
  args.integration_limit = .3
  force_recalc = False

  resnet_checkpoint = f"snapshots/{args.category}/best.pth.tar"
  resnet = d.load_STM(resnet_checkpoint,
                      model_type='resnet18',
                      # category=args.category,
                      aggregator='prod',
                      force_recalc=force_recalc)
  resnet.cuda()

  auc, img_auc, pro = d.test_with_model_and_args(resnet, args)
  aucs.append(auc)
  img_aucs.append(img_auc)
  pros.append(pro)

df_resnet_STM = pd.DataFrame(data={'Category': d.MVTEC_ITEMTYPES + ['mean'],
                        'AUROC': aucs + [np.mean(aucs)],
                        'Image-level': img_aucs + [np.mean(img_aucs)],
                        # 'PRO': pros + [np.mean(pros)],
                        })

df_resnet_STM

Category: carpet	Pixel-AUC: 0.990211	Image-AUC: 0.963884
Category: grid	Pixel-AUC: 0.989774	Image-AUC: 0.982456
Category: leather	Pixel-AUC: 0.990985	Image-AUC: 0.950068
Category: tile	Pixel-AUC: 0.969538	Image-AUC: 0.981962
Category: wood	Pixel-AUC: 0.964892	Image-AUC: 0.995614
Category: bottle	Pixel-AUC: 0.988124	Image-AUC: 1.000000
Category: cable	Pixel-AUC: 0.957285	Image-AUC: 0.936657
Category: capsule	Pixel-AUC: 0.986065	Image-AUC: 0.894695
Category: hazelnut	Pixel-AUC: 0.984909	Image-AUC: 1.000000
Category: metal_nut	Pixel-AUC: 0.971815	Image-AUC: 0.983871
Category: pill	Pixel-AUC: 0.975147	Image-AUC: 0.950900
Category: screw	Pixel-AUC: 0.988345	Image-AUC: 0.895880
Category: toothbrush	Pixel-AUC: 0.989824	Image-AUC: 0.897222
Category: transistor	Pixel-AUC: 0.819428	Image-AUC: 0.940000
Category: zipper	Pixel-AUC: 0.987908	Image-AUC: 0.960872


,Category,AUROC,Image-level
0,carpet,0.990211,0.963884
1,grid,0.989774,0.982456
2,leather,0.990985,0.950068
3,tile,0.969538,0.981962
4,wood,0.964892,0.995614
5,bottle,0.988124,1.000000
6,cable,0.957285,0.936657
7,capsule,0.986065,0.894695
8,hazelnut,0.984909,1.000000
9,metal_nut,0.971815,0.983871


In [6]:
# Now use the same ResNet-18 model, but with discrepancy scaling enabled.

aucs = []
img_aucs = []
pros = []

for category in d.MVTEC_ITEMTYPES:
  args = d.DummyArgs()
  args.category = category
  args.scaling = True
  args.eps = 0.05
  args.pro = False
  args.data_dir = DATASET_PATH
  args.batch_size = 32
  args.integration_limit = .3
  force_recalc = True

  resnet_checkpoint = f"snapshots/{args.category}/best.pth.tar"
  resnet = d.load_STM(resnet_checkpoint,
                      model_type='resnet18',
                      category=args.category,
                      aggregator='prod',
                      force_recalc=force_recalc)
  resnet.cuda()

  auc, img_auc, pro = d.test_with_model_and_args(resnet, args)
  aucs.append(auc)
  img_aucs.append(img_auc)
  pros.append(pro)

df_resnet_scaling = pd.DataFrame(data={'Category': d.MVTEC_ITEMTYPES + ['mean'],
                        'AUROC': aucs + [np.mean(aucs)],
                        'Image-level': img_aucs + [np.mean(img_aucs)],
                        # 'PRO': pros + [np.mean(pros)],
                        })

df_resnet_scaling

Category: carpet	Pixel-AUC: 0.991735	Image-AUC: 0.989968
Category: grid	Pixel-AUC: 0.991092	Image-AUC: 0.994152
Category: leather	Pixel-AUC: 0.995648	Image-AUC: 1.000000
Category: tile	Pixel-AUC: 0.970946	Image-AUC: 0.984848
Category: wood	Pixel-AUC: 0.965026	Image-AUC: 0.994737
Category: bottle	Pixel-AUC: 0.989613	Image-AUC: 1.000000
Category: cable	Pixel-AUC: 0.965239	Image-AUC: 0.929160
Category: capsule	Pixel-AUC: 0.987236	Image-AUC: 0.914639
Category: hazelnut	Pixel-AUC: 0.984065	Image-AUC: 0.967857
Category: metal_nut	Pixel-AUC: 0.979059	Image-AUC: 0.910557
Category: pill	Pixel-AUC: 0.973357	Image-AUC: 0.936716
Category: screw	Pixel-AUC: 0.988639	Image-AUC: 0.894651
Category: toothbrush	Pixel-AUC: 0.987588	Image-AUC: 0.858333
Category: transistor	Pixel-AUC: 0.860850	Image-AUC: 0.936667
Category: zipper	Pixel-AUC: 0.989671	Image-AUC: 0.962185


,Category,AUROC,Image-level
0,carpet,0.991735,0.989968
1,grid,0.991092,0.994152
2,leather,0.995648,1.000000
3,tile,0.970946,0.984848
4,wood,0.965026,0.994737
5,bottle,0.989613,1.000000
6,cable,0.965239,0.929160
7,capsule,0.987236,0.914639
8,hazelnut,0.984065,0.967857
9,metal_nut,0.979059,0.910557


In [ ]:
# We can now train a second ResNet-18 for ensembling

for cat in d.MVTEC_ITEMTYPES:

  print(f"Training a model for category {cat}")

  np.random.seed(1)
  torch.manual_seed(1)

  args = d.DummyArgs()
  args.data_dir = DATASET_PATH
  args.batch_size = 32
  args.category = cat
  args.epochs = 200
  args.model_save_path = "snapshots"

  teacher = d.ResNet18_MS3(pretrained=True)
  student = d.ResNet18_MS3(pretrained=False)
  teacher.cuda()
  student.cuda()
  model_name = "resnet18_1"

  train_loader, val_loader = d.get_train_val_loaders(cat)

  d.train_val(teacher, student, train_loader, val_loader, args, model_name=model_name)

In [10]:
# Train a MobileNet v2 for each itemtype
for category in d.MVTEC_ITEMTYPES:
  print(f"Training a MobileNetV2 for category {category}")
  teacher_mnet = d.MNet(pretrained=True)
  student_mnet = d.MNet(pretrained=False)
  teacher_mnet.cuda()
  student_mnet.cuda()
  train_loader, val_loader = d.get_train_val_loaders(category=category)
  args = d.DummyArgs()
  args.data_dir = DATASET_PATH
  args.batch_size = 32
  args.category = category
  args.epochs = 200
  args.model_save_path = "snapshots"

  d.train_val(teacher_mnet,
            student_mnet,
            train_loader,
            val_loader,
            args,
            model_name='mnet')

Training a MobileNetV2 for category carpet
[0/200] loss: 10.005419
[0/200] loss: 9.798120
[0/200] loss: 9.488725
[0/200] loss: 9.058526
[0/200] loss: 8.563688
[0/200] loss: 8.009922
[0/200] loss: 7.418671
Validation Loss: 25.3016574
[1/200] loss: 6.842853
[1/200] loss: 6.281956
[1/200] loss: 5.762999
[1/200] loss: 5.261139
[1/200] loss: 4.839705
[1/200] loss: 4.429299
[1/200] loss: 4.023038
Validation Loss: 22.0375250
[2/200] loss: 3.659973
[2/200] loss: 3.404284
[2/200] loss: 3.076950
[2/200] loss: 2.854985
[2/200] loss: 2.697964
[2/200] loss: 2.510096
[2/200] loss: 2.278586
Validation Loss: 9.8958761
[3/200] loss: 2.145125
[3/200] loss: 1.992219
[3/200] loss: 1.921592
[3/200] loss: 1.842482
[3/200] loss: 1.840953
[3/200] loss: 1.660418
[3/200] loss: 1.624499
Validation Loss: 2.8716283
[4/200] loss: 1.523907
[4/200] loss: 1.515469
[4/200] loss: 1.493134
[4/200] loss: 1.395677
[4/200] loss: 1.359396
[4/200] loss: 1.371398
[4/200] loss: 1.327848
Validation Loss: 0.4452468
[5/200] loss: 

In [4]:
# Ensemble two ResNet-18s
# Use discrepancy scaling

aucs = []
img_aucs = []
pros = []

for category in d.MVTEC_ITEMTYPES:
  args = d.DummyArgs()
  args.category = category
  args.scaling = True
  args.pro = False
  args.data_dir = DATASET_PATH
  args.batch_size = 32
  args.integration_limit = .3
  force_recalc = False

  resnet1_checkpoint = f"snapshots/{args.category}/best.pth.tar"
  resnet1 = d.load_STM(resnet1_checkpoint,
                    model_type='resnet18',
                    category=args.category,
                    aggregator='prod',
                    force_recalc=force_recalc)
  resnet1.eps = 0.05

  resnet2_checkpoint = f"snapshots/{args.category}/resnet18_1.pth.tar"
  resnet2 = d.load_STM(resnet2_checkpoint,
                    model_type='resnet18',
                    category=args.category,
                    aggregator='prod',
                    force_recalc=force_recalc)
  resnet2.eps = 0.05

  _, val_loader = d.get_train_val_loaders(args.category)

  ensemble_wts = torch.tensor([.5,.5])
  ensemble = d.NewEnsemble([resnet1,
                          resnet2],
                          weights=ensemble_wts,
                          val_loader=val_loader
                          )
  ensemble.eval()                 
  ensemble.cuda()                    

  auc, img_auc, pro = d.test_with_model_and_args(ensemble, args)
  aucs.append(auc)
  img_aucs.append(img_auc)
  pros.append(pro)

df_two_resnet_ensemble = pd.DataFrame(data={'Category': d.MVTEC_ITEMTYPES + ['mean'],
                        'AUROC': aucs + [np.mean(aucs)],
                        'Image-level': img_aucs + [np.mean(img_aucs)],
                        # 'PRO': pros + [np.mean(pros)],
                        })

df_two_resnet_ensemble

Category: carpet	Pixel-AUC: 0.990901	Image-AUC: 0.963082
Category: grid	Pixel-AUC: 0.991680	Image-AUC: 0.996658
Category: leather	Pixel-AUC: 0.995410	Image-AUC: 1.000000
Category: tile	Pixel-AUC: 0.971786	Image-AUC: 0.981962
Category: wood	Pixel-AUC: 0.970665	Image-AUC: 0.994737
Category: bottle	Pixel-AUC: 0.989649	Image-AUC: 1.000000
Category: cable	Pixel-AUC: 0.965049	Image-AUC: 0.938156
Category: capsule	Pixel-AUC: 0.986871	Image-AUC: 0.906661
Category: hazelnut	Pixel-AUC: 0.983533	Image-AUC: 0.981429
Category: metal_nut	Pixel-AUC: 0.979899	Image-AUC: 0.978495
Category: pill	Pixel-AUC: 0.977007	Image-AUC: 0.942717
Category: screw	Pixel-AUC: 0.988243	Image-AUC: 0.895675
Category: toothbrush	Pixel-AUC: 0.987442	Image-AUC: 0.847222
Category: transistor	Pixel-AUC: 0.860792	Image-AUC: 0.948750
Category: zipper	Pixel-AUC: 0.989363	Image-AUC: 0.954832


,Category,AUROC,Image-level
0,carpet,0.990901,0.963082
1,grid,0.991680,0.996658
2,leather,0.995410,1.000000
3,tile,0.971786,0.981962
4,wood,0.970665,0.994737
5,bottle,0.989649,1.000000
6,cable,0.965049,0.938156
7,capsule,0.986871,0.906661
8,hazelnut,0.983533,0.981429
9,metal_nut,0.979899,0.978495
